<a href="https://colab.research.google.com/github/Shantesh123/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, concatenate
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [6]:
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [11]:
X_columns = ['Age', 'DistanceFromHome', 'Education', 'JobLevel', 'HourlyRate', 'YearsAtCompany', 'TotalWorkingYears', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']
X_df = attrition_df[X_columns]
print("Data types for X_df:")
print(X_df.dtypes)

Data types for X_df:
Age                        int64
DistanceFromHome           int64
Education                  int64
JobLevel                   int64
HourlyRate                 int64
YearsAtCompany             int64
TotalWorkingYears          int64
YearsInCurrentRole         int64
YearsSinceLastPromotion    int64
YearsWithCurrManager       int64
dtype: object


In [13]:
num_input_cols = X_df.shape[1]
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [14]:
# Convert X data to numeric types
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
encoder_dept = OneHotEncoder()
dept_train_encoded = encoder_dept.fit_transform(y_train[['Department']]).toarray()
dept_test_encoded = encoder_dept.transform(y_test[['Department']]).toarray()

In [25]:
# One-hot encode the attrition column
encoder_attrition = OneHotEncoder()
attrition_train_encoded = encoder_attrition.fit_transform(y_train[['Attrition']]).toarray()
attrition_test_encoded = encoder_attrition.transform(y_test[['Attrition']]).toarray()

In [20]:
# Create the input layer
input_layer = Input(shape=(num_input_cols,))

In [21]:
# Create shared layers
shared_layer1 = Dense(64, activation='relu')
shared_layer2 = Dense(32, activation='relu')

In [22]:
# Create branches for department and attrition prediction
department_branch = shared_layer2(shared_layer1(input_layer))
department_output = Dense(dept_train_encoded.shape[1], activation='softmax', name='department_output')(department_branch)

In [26]:
attrition_branch = shared_layer2(shared_layer1(input_layer))
attrition_output = Dense(attrition_train_encoded.shape[1], activation='softmax', name='attrition_output')(attrition_branch)

In [27]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

In [29]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summarize the model
model.summary()

# Train the model
model.fit(X_train_scaled, {'department_output': dept_train_encoded, 'attrition_output': attrition_train_encoded}, epochs=10, batch_size=32)

# Evaluate the model
department_accuracy, attrition_accuracy = model.evaluate(X_test_scaled, {'department_output': dept_test_encoded, 'attrition_output': attrition_test_encoded})
print(f"Department Accuracy: {department_accuracy}")
print(f"Attrition Accuracy: {attrition_accuracy}")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 64)                   704       ['input_1[0][0]',             
                                                                     'input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 32)                   2080      ['dense[0][0]',               
                                                                     'dense[2][0]']               
                                                                                              

ValueError: too many values to unpack (expected 2)

Is accuracy the best metric to use on this data? Why or why not?

While accuracy is a commonly used metric, it may not be the best choice for imbalanced datasets like the one with attrition data. In cases of imbalanced classes, metrics like precision, recall, F1-score, or area under the ROC curve may provide a more comprehensive evaluation of the model's performance.

What activation functions did you choose for your output layers, and why?

For the output layers predicting categorical variables like department and attrition, I used the softmax activation function. Softmax is suitable for multi-class classification tasks as it converts the output values into probabilities, making it easier to interpret and compare the predicted classes.

Can you name a few ways that this model could be improved?

Increase model complexity by adding more layers or neurons to capture intricate relationships in the data.
Implement regularization techniques like dropout or L2 regularization to prevent overfitting.
Experiment with different optimizer functions or learning rates to improve convergence speed.
Perform hyperparameter tuning to optimize model performance.
Consider incorporating feature engineering or additional data preprocessing steps to enhance model input.